In [1]:
%reload_ext autoreload

# Imports

import os
import sys
module_path = os.path.abspath(os.path.join('../image_gen_dm')) # or the path to your source code
sys.path.append(str(module_path))

import tensorflow as tf
import tf_keras as tfk
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

tfkl = tfk.layers
tfpl = tfp.layers
tfd = tfp.distributions

TF_ENABLE_ONEDNN_OPTS=0
os.environ['TF_ENABLE_ONEDNN_OPTS'] = str(TF_ENABLE_ONEDNN_OPTS)
os.environ['TG_GPU_ALLOCATOR'] = 'cuda_malloc_async'

import image_gen_vae as igvae
import image_gen_vae.constants as consts

print('Tensorflow Version:', tf.__version__)
print("Available GPUs: ", len(tf.config.list_physical_devices('GPU')))

2024-04-28 13:04:39.996095: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Tensorflow Version: 2.16.1
Available GPUs:  1


2024-04-28 13:04:46.924733: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-28 13:04:47.005697: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-28 13:04:47.005747: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [6]:
# Constants

IMAGE_SIZE = 64
FLATTENED_SIZE = IMAGE_SIZE * IMAGE_SIZE * 3

ENCODER_HIDDEN_SIZES_VALUES = [
    [7168, 8192, 9216],
    [1792, 2048, 2304],
    [448, 512, 576]
]

ENCODER_HIDDEN_SIZES = []

for size1 in ENCODER_HIDDEN_SIZES_VALUES[0]:
    for size2 in ENCODER_HIDDEN_SIZES_VALUES[1]:
        for size3 in ENCODER_HIDDEN_SIZES_VALUES[2]:
            ENCODER_HIDDEN_SIZES.append([size1, size2, size3])

print(f'Encoder Hidden Sizes: {len(ENCODER_HIDDEN_SIZES)}')
print(ENCODER_HIDDEN_SIZES)

DECODER_HIDDEN_SIZES = [] # Construct from encoder hidden sizes in reverse

for hidden_sizes in ENCODER_HIDDEN_SIZES:
    DECODER_HIDDEN_SIZES.append(hidden_sizes[::-1])

MODEL_NAMES = []

for i in range(len(ENCODER_HIDDEN_SIZES)):
    # +14 to avoid overwriting the previous models
    MODEL_NAMES.append(f'ncvae{i+1+14}')

    if MODEL_NAMES[-1] == 'ncvae24':
        print(ENCODER_HIDDEN_SIZES[i])

LATENT_DIM = 2

LEARNING_RATE = 1e-4

VAL_PERCENTAGE = 0.1

# Models to run

MODEL_START = 9 # Rasmus 0-14, Viktor 15-26
MODEL_COUNT = 1

Encoder Hidden Sizes: 27
[[7168, 1792, 448], [7168, 1792, 512], [7168, 1792, 576], [7168, 2048, 448], [7168, 2048, 512], [7168, 2048, 576], [7168, 2304, 448], [7168, 2304, 512], [7168, 2304, 576], [8192, 1792, 448], [8192, 1792, 512], [8192, 1792, 576], [8192, 2048, 448], [8192, 2048, 512], [8192, 2048, 576], [8192, 2304, 448], [8192, 2304, 512], [8192, 2304, 576], [9216, 1792, 448], [9216, 1792, 512], [9216, 1792, 576], [9216, 2048, 448], [9216, 2048, 512], [9216, 2048, 576], [9216, 2304, 448], [9216, 2304, 512], [9216, 2304, 576]]
[8192, 1792, 448]


In [ ]:
# Dataset loading

train_ds, val_ds = igvae.utils.load_datasets(val_percentage=VAL_PERCENTAGE)

In [ ]:
# Model class

# This code is heavily based on the Keras Team's example on Convolutional VAEs with only minor modifications.
# The original code can be found at: https://github.com/keras-team/keras-io/blob/master/examples/generative/vae.py

# Author: [fchollet](https://twitter.com/fchollet)
# Licensed under the Apache License 2.0. See LICENSE file in the root directory for more information.


class Sampling(tfkl.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

class NCVAE(tfk.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = tfk.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = tfk.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = tfk.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]
    
    def train_step(self, data):
        with tf.GradientTape() as tape:
            input_data, target_data = data
            z_mean, z_log_var, z = self.encoder(input_data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    tf.losses.binary_crossentropy(target_data, reconstruction), 
                    axis=(1,2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss

        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }
    
    def call(self, input_data, training=False):
        _, _, z = self.encoder(input_data)
        reconstructed_image = self.decoder(z)

        return reconstructed_image


In [ ]:
# Encoder

models = []

for i in range(MODEL_START, MODEL_START + MODEL_COUNT):
    print(f'Model {i+1+14}')

    image_input = tfkl.Input(shape=consts.INPUT_SHAPE, name='image_input')

    flattened_image = tfkl.Flatten()(image_input)

    x = flattened_image
    for layer_size in ENCODER_HIDDEN_SIZES[i]:
        x = tfkl.Dense(layer_size, 
                    activation='relu',
                    kernel_initializer='glorot_uniform')(x)
        
    z_mean = tfkl.Dense(LATENT_DIM, name='z_mean', kernel_initializer='glorot_uniform')(x)
    z_log_var = tfkl.Dense(LATENT_DIM, name='z_log_var', kernel_initializer='glorot_uniform')(x)

    z = Sampling()([z_mean, z_log_var])

    encoder = tfk.Model(inputs=image_input, outputs=[z_mean, z_log_var, z], name='encoder')
    encoder.summary()

    # Decoder

    latent_inputs = tfkl.Input(shape=(LATENT_DIM,))

    y = latent_inputs
    for layer_size in DECODER_HIDDEN_SIZES[i]:
        y = tfkl.Dense(layer_size, 
                    activation='relu',
                    kernel_initializer='glorot_uniform')(y)
        
    reconstructed_flattened_image = tfkl.Dense(FLATTENED_SIZE, kernel_initializer='glorot_uniform', activation='sigmoid')(y)
    reshaped_image = tfkl.Reshape(consts.OUTPUT_SHAPE)(reconstructed_flattened_image)

    decoder = tfk.Model(inputs=latent_inputs, outputs=reshaped_image, name='decoder')
    decoder.summary()

    # Model
        
    model = NCVAE(encoder, decoder)
    model.compile(optimizer=tfk.optimizers.Adam(learning_rate=LEARNING_RATE))

    models.append(model)

In [ ]:
# Run model

epochs = 256

i = MODEL_START
for model in models:

    history = igvae.run_model(model=model, train_ds=train_ds, val_ds=val_ds, epochs=epochs, name=MODEL_NAMES[i], plot=False)
    
    pd.DataFrame(history.history).to_csv(f'res/histories/{MODEL_NAMES[i]}.csv')

    i += 1
